# Exibição de informações dos parlamentares

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql

import seaborn as sns
import csv
import squarify 

import os

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

## Carga de infos básicas. 

In [2]:
#!ls ../dados/tse/ibg

### Carga de parlamentares

In [3]:
#Dados de parlamentares
con = sql.connect("../dados/sql/base_completa.db")
df_parlamentares=pd.read_sql("select * from cadastro",con)
con.close()

In [4]:
df_parlamentares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CPF                      592 non-null    object 
 1   NM_CANDIDATO             592 non-null    object 
 2   DT_NASCIMENTO            592 non-null    object 
 3   SG_UF_NASCIMENTO         592 non-null    object 
 4   NM_MUNICIPIO_NASCIMENTO  592 non-null    object 
 5   DS_GENERO                592 non-null    object 
 6   DS_GRAU_INSTRUCAO        592 non-null    object 
 7   DS_ESTADO_CIVIL          592 non-null    object 
 8   DS_COR_RACA              592 non-null    object 
 9   DS_OCUPACAO              592 non-null    object 
 10  VL_BENS                  592 non-null    float64
 11  ANO_ELEICAO              592 non-null    int64  
 12  DS_CARGO                 592 non-null    object 
 13  SG_PARTIDO               592 non-null    object 
 14  SG_UE                    5

## Carga de municipios e votação

In [5]:
#Dados de municípios
con = sql.connect("../dados/sql/base_completa.db")
df_municipios_votacao=pd.read_sql("select * from parl_votacao",con)
df_municipios=pd.read_sql("select * from municipios",con)
con.close()

In [6]:
#df_municipios.columns[18]
#df_municipios.head()
df_municipios_votacao.info()
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191313 entries, 0 to 191312
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CPF              191313 non-null  object 
 1   SG_UF            191313 non-null  object 
 2   CD_MUNICIPIO     191313 non-null  int64  
 3   VOTOS_TOTAL_MUN  191313 non-null  int64  
 4   CODIGO_IBGE      190671 non-null  float64
 5   PERC_VOTOS       191313 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 8.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODIGO_IBGE       5546 non-null   float64
 1   SG_UF             5570 non-null   object 
 2   NM_MUNICIPIO      5570 non-null   object 
 3   IDHM_2010         5546 non-null   float64
 4   IDHM_RENDA_2010   5546 non-null   float64
 5   IDHM_LONGEV_

## Carga de informações geograficas dos municípios

In [7]:
# informação geográfica dos municípios
lista_uf=list(df_municipios_votacao["SG_UF"].unique())
df_mapa_brasil=pd.DataFrame()

In [8]:
for uf in lista_uf:
    diretorio="../dados/ibge/shapes/"+uf
    arquivos=os.listdir(diretorio)
    shapefile = [arq for arq in arquivos if ".shp" in arq][0]
    arq=diretorio+"/"+shapefile
    df_uf= gpd.read_file(arq)
    df_uf["SG_UF"]=uf
    df_mapa_brasil=pd.concat([df_mapa_brasil,df_uf])
    

In [9]:
#df_mapa_brasil.info()

In [10]:
df_mapa_brasil=df_mapa_brasil.astype({"CD_GEOCODM":"float64"})
df_mapa_brasil.rename(columns={"CD_GEOCODM":"CODIGO_IBGE"}, inplace=True)
#df_mapa_brasil.info()

In [11]:
#df_mapa_brasil.head()
#df_mapa_brasil.to_csv("../dados/ibge/mapa_brasil.csv")

# Funções de exibição

In [12]:
# Plot estado e municipios com indicador
def plot_estado(estado,indicador,titulo, minimo,maximo, cor):
    vmin=minimo
    vmax=maximo
    fig, ax = plt.subplots(figsize=(12,8))
    estado.plot(ax=ax, column=indicador, cmap=cor, edgecolor="black", linewidth=0.2)
    sm = plt.cm.ScalarMappable(cmap=cor, norm=plt.Normalize(vmin=vmax, vmax=vmin))
    sm._A = []
    cbar = fig.colorbar(sm)
    plt.title(titulo, fontsize=40, color="grey")
    ax.axis('off')
    plt.axis("equal")
    plt.show()
    return

In [13]:
def plot_treemap(estado,indicador,titulo, minimo,maximo, cor):
    estado["NOME_IDH"]=estado["NM_MUNICIPIO"]+"\n IDH - "+estado["IDHM_2010"].round(6).astype(str)+"\n Votos - "+(estado["PERC_VOTOS"]*100).round(2).astype(str)+"%"
    vmin=maximo
    vmax=minimo
    cmap = plt.cm.RdBu
    norm = plt.Normalize(vmin=vmax, vmax=vmin)
    colors = [cmap(norm(value)) for value in estado[indicador]]
    fig, ax = plt.subplots(figsize=(16,12))
    sm = plt.cm.ScalarMappable(cmap=cor, norm=plt.Normalize(vmin=vmax, vmax=vmin))
    sm._A = []
    cbar = fig.colorbar(sm)
    plt.title(titulo, fontsize=40, color="grey")
    ax.axis('off')
    plt.axis("equal")
    squarify.plot(ax=ax, sizes=estado['VOTOS_TOTAL_MUN'][0:20], 
                  label=estado['NOME_IDH'][0:20], 
                  color=colors[0:20],
                  alpha=.8 )
    

In [14]:
def exibe_cadastro(cpf):
    df_parlamentar=df_parlamentares[df_parlamentares["CPF"]==cpf]
    valor_bens=df_parlamentar["VL_BENS"]
    campos=["NM_URNA_CANDIDATO","NM_CANDIDATO","DT_NASCIMENTO","DS_COR_RACA",
            "DS_GRAU_INSTRUCAO","DS_OCUPACAO"]
    parlamentar=df_parlamentar[campos].to_dict(orient="records")[0]
    parlamentar["BENS"]=valor_bens.round(2)
    print(parlamentar)
    return


In [15]:
def exibe_votacao_estado(cpf, cpf_votacao):
    uf=cpf_votacao["SG_UF"].iloc[0]
    nome=cpf["NM_URNA_CANDIDATO"].iloc[0]
    cpf_municipios=df_mapa_brasil[df_mapa_brasil["SG_UF"]==uf]
    cpf_municipios=pd.merge(cpf_municipios,cpf_votacao, how="left", on="CODIGO_IBGE")
    cpf_municipios=pd.merge(cpf_municipios,df_municipios, how="left", on="CODIGO_IBGE")
    cpf_municipios=cpf_municipios.sort_values('VOTOS_TOTAL_MUN', ascending=False)
    indicador=indicador="IDHM_2010"
    print(cpf_municipios.columns)
    titulo=uf+" - "+"IDHM Municípios"
    minimo=0
    maximo=1
    cor="RdBu"
    plot_estado(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    indicador="VOTOS_TOTAL_MUN"
    titulo=nome+" - "+"Votos por município"
    minimo=0
    maximo=cpf_municipios[indicador].max()
    cor="Blues"
    plot_estado(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    indicador=indicador="IDHM_2010"
    titulo=nome+" - "+"IDHM e Votos por município"
    minimo=0
    maximo=1
    cor="RdBu"
    plot_treemap(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    return

# Escolhe randomicamente um CPF e chama as funções

In [40]:
lista_1=df_parlamentares["SG_UE"].unique()
lista_1
lista_2=list(lista_1)
lista_2
lista_3=sorted(lista_2)
lista_3

array(['MT', 'SP', 'DF', 'PR', 'GO', 'RJ', 'PE', 'AP', 'SC', 'ES', 'RO',
       'AM', 'CE', 'AL', 'RN', 'PI', 'AC', 'SE', 'TO', 'MS', 'MA', 'RR',
       'PA', 'BA', 'PB', 'RS', 'MG'], dtype=object)

['MT',
 'SP',
 'DF',
 'PR',
 'GO',
 'RJ',
 'PE',
 'AP',
 'SC',
 'ES',
 'RO',
 'AM',
 'CE',
 'AL',
 'RN',
 'PI',
 'AC',
 'SE',
 'TO',
 'MS',
 'MA',
 'RR',
 'PA',
 'BA',
 'PB',
 'RS',
 'MG']

['AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO']

In [ ]:
cpf=df_parlamentares["CPF"].sample(n=1).iloc[0]
cpf
df_cpf_votacao=df_municipios_votacao[df_municipios_votacao["CPF"]==cpf]
df_cpf=df_parlamentares[df_parlamentares["CPF"]==cpf]
exibe_cadastro(cpf)
exibe_votacao_estado(df_cpf, df_cpf_votacao)

In [ ]:
#df['Volume'] = df.Length*df.Height*df.Depth
#df.assign(Area=lambda df: df.Length*df.Height)
#f["period"] = df["Year"] + df["quarter"]
df_cpf_votacao.head()
x=pd.DataFrame()
df_cpf_votacao["CD_PERC"]=df_cpf_votacao["SG_UF"]+df_cpf_votacao["PERC_VOTOS"].round(6).astype(str)
#+str(df_cpf_votacao["PERC_VOTOS"])
df_cpf_votacao.head()

In [ ]:
df_cpf
df_cpf_votacao.head()

In [ ]:
df_parlamentares.columns

In [ ]:
df_cpf_municipios.columns
df_municipios=df_cpf_municipios.sort_values("NM_MUNICIPIO").head()
df_cpf_municipios.shape

In [ ]:
df_cpf_votacao=df_cpf_municipios[["NM_MUNICIP","QT_VOTOS_NOMINAIS"]].sort_values("QT_VOTOS_NOMINAIS", 
                                                                                 ascending=False)

In [ ]:
df_cpf_votacao.head(10)
df_cpf_votacao.shape

In [ ]:
total=df_cpf_votacao["QT_VOTOS_NOMINAIS"].sum()
df_cpf_votacao["PERCENT"]=df_cpf_votacao["QT_VOTOS_NOMINAIS"]/total

In [ ]:
df_cpf_votacao.head()
df_cpf_votacao.shape
total

In [ ]:
import streamlit as st

In [ ]:
# libraries
import matplotlib.pyplot as plt
import squarify    # pip install squarify (algorithm for treemap)
 
# If you have 2 lists
squarify.plot(sizes=[13,22,35,5], label=["group A", "group B", "group C", "group D"], alpha=.7 )
plt.axis('off')
plt.show()
 
# If you have a data frame?
import pandas as pd
df = pd.DataFrame({'nb_people':[8,3,4,2], 'group':["group A", "group B", "group C", "group D"] })
squarify.plot(sizes=df['nb_people'], label=df['MUNICIPIO'], alpha=.8 )
plt.axis('off')
plt.show() 

In [ ]:
#libraries
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import squarify # pip install squarify (algorithm for treemap)&lt;/pre&gt;
 
# Create a dataset:
my_values=[i**3 for i in range(1,100)]
 
# create a color palette, mapped to these values
cmap = matplotlib.cm.Blues
mini=min(my_values)
maxi=max(my_values)
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in my_values]
 
# Change color
squarify.plot(sizes=my_values, alpha=.8, color=colors )
plt.axis('off')
plt.show()